In [10]:
from elasticsearch import Elasticsearch
from keras.models import Model,load_model
from nltk.tokenize import sent_tokenize
import nltk
import numpy as np
from keras.preprocessing import text, sequence
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
model = load_model('my_model.h5')

In [38]:
ES_settings={
    'ES_hosts':'ec2-52-34-223-218.us-west-2.compute.amazonaws.com',
'ES_user':'elastic',
'ES_password':'elasticdevpass',
    'ES_type':'plainDocs',
    'ES_index':'crawls'
}

In [51]:
hosts=[ES_settings['ES_hosts']]
es = Elasticsearch(
       	    		hosts,
            		port=9200,
	    		http_auth=(ES_settings['ES_user'], ES_settings['ES_password']), 
	    		verify_certs=False,
            		sniff_on_start=True,    # sniff before doing anything
            		sniff_on_connection_fail=True,    # refresh nodes after a node fails to respond
            		sniffer_timeout=60, # and also every 60 seconds
            		timeout=15

)

In [52]:
search_text='cloning'

In [53]:
response = es.search(index=ES_settings['ES_index'], body={
            "query": {
                "match": {
                    "doc": search_text
                    }
                    } })


if response['timed_out'] == True :
                jsonresponse =[]


else:
        search_results =   response['hits']['hits']
        total_took = response['took']


In [54]:
max_features=1000
maxlen=150
embed_size=300
tok=text.Tokenizer(num_words=max_features,lower=True)


In [61]:
all_sentences=[]
x=0
for X in search_results:
    plaintext=X['_source']['doc'] 
    print(plaintext)
    print('/n')
    all_sentences+=sent_tokenize(plaintext)
print (len(all_sentences))
tok.fit_on_texts(all_sentences)
input_test=tok.texts_to_sequences(all_sentences)
X_test=sequence.pad_sequences(input_test, maxlen=maxlen)
y_pred = model.predict(X_test,batch_size=1024,verbose=1)
index=np.arange(len(y_pred))
y_pred=np.c_[ index,y_pred ]  
#np.append(y_pred, index, axis=1)


GAINESVILLE, FL – Scientists at the University of Florida have successfully cloned a dinosaur, a spokesman from the university said yesterday.GAINESVILLE, FL – Scientists at the University of Florida have successfully cloned a dinosaur, a spokesman from the university said yesterday.The dinosaur, a baby Apatosaurus nicknamed “Spot,” is currently being incubated at the University of Florida’s College of Veterinary Medicine.The scientists extracted DNA from preserved Apatosaurus fossils, which were on display at the university’s museum of natural science. Once the DNA was harvested, scientists injected it into a fertile ostrich womb.“Ostriches share a lot of genetic traits with dinosaurs,” said Dr. Norman Trudell, a biology professor at UF and the project’s leading scientist. “Their eggshell microstructures are almost identical to those of the Apatosaurus. That’s why the cloning worked so perfectly.”Those in the scientific community say the dinosaur cloning – the first ever of its kind –

207/207 [==============================] - 0s 1ms/step


In [56]:
ratings = pd.DataFrame({'index':y_pred[:,0],'no_Argument':y_pred[:,1],'Argument_for':y_pred[:,2],'Argument_against':y_pred[:,3]})


In [57]:
def compare(no_Argument,Argument_for,Argument_against):
    maxval=max(no_Argument,Argument_for,Argument_against)
    if maxval==no_Argument:
        return 'no_Argument'
    elif maxval==Argument_for:
        return 'Argument_for'
    return 'Argument_against'

In [58]:
ratings['res']=ratings.apply(lambda x: compare(x['no_Argument'],x['Argument_for'],x['Argument_against']),axis=1)

In [59]:
def print_argument(x):
    if x['res']!='no_Argument':
        print ("\n")
        print ("*****")
        print (x['res'])
        print (all_sentences[(int(x['index']))])
    return 

In [60]:
ratings.apply(lambda x:print_argument(x),axis=1)



*****
Argument_against
GAINESVILLE, FL – Scientists at the University of Florida have successfully cloned a dinosaur, a spokesman from the university said yesterday.GAINESVILLE, FL – Scientists at the University of Florida have successfully cloned a dinosaur, a spokesman from the university said yesterday.The dinosaur, a baby Apatosaurus nicknamed “Spot,” is currently being incubated at the University of Florida’s College of Veterinary Medicine.The scientists extracted DNA from preserved Apatosaurus fossils, which were on display at the university’s museum of natural science.


*****
Argument_against
Once the DNA was harvested, scientists injected it into a fertile ostrich womb.“Ostriches share a lot of genetic traits with dinosaurs,” said Dr. Norman Trudell, a biology professor at UF and the project’s leading scientist.


*****
Argument_for
That’s why the cloning worked so perfectly.”Those in the scientific community say the dinosaur cloning – the first ever of its kind – is a miles

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
177    None
178    None
179    None
180    None
181    None
182    None
183    None
184    None
185    None
186    None
187    None
188    None
189    None
190    None
191    None
192    None
193    None
194    None
195    None
196    None
197    None
198    None
199    None
200    None
201    None
202    None
203    None
204    None
205    None
206    None
Length: 207, dtype: object